# Eddy Viscosity Models and Solver Review

**Here's a brief review of the solver and the eddy viscosity models:**

-  This notebook includes an eddy viscosity model where the *eddy viscosity coefficient* can be dynamically adjusted.


## 2D Turbulence

We solve Eqs. in the vorticity-streamfunction, $\omega-\psi$, formulation, where



With this formulation, the governing equations are

$$
\begin{aligned}
\nabla^2\psi &= - \omega, \tag{1} \\
\frac{\partial \omega}{\partial t} + \mathcal{N}\left(\omega,\psi\right) &= \frac{1}{Re}\nabla^2\omega -\chi \omega - f + \beta v,
\end{aligned}
$$

where $\mathcal{N}(\omega,\psi)$ is 

$$
\mathcal{N}(\omega,\psi) = \frac{\partial \omega}{\partial x}\frac{\partial \psi}{\partial y} - \frac{\partial \omega}{\partial y}\frac{\partial \psi}{\partial x}. 
$$

$f$ is a deterministic forcing \cite{chandler2013invariant, kochkov2021machine}:

$$
f = f_{k_x}\cos(f_{k_x}x) + f_{k_y}\cos(f_{k_y}y), \tag{4}
$$

where $f_{k_{x}}$ and $f_{k_{y}}$ are the forcing wavenumbers, $\chi$ represents the Rayleigh drag coefficient, $\beta$ represents the effect of beta-plane turbulence, and $\bm{u}=(u,v)$ is the velocity. In 2D, the velocity can be represented by stream function

$$
u = \frac{\partial \psi}{\partial y}, v = -\frac{\partial \psi}{\partial x},
$$

In DNS, governing equations are solved in a doubly periodic domain using a Fourier-Fourier pseudo-spectral solver with second-order Adams-Bashforth and Crank Nicholson time-integration schemes for the advection and viscous terms, respectively (time step $\Delta t_{\text{DNS}}$).

For LES, we use the same solver with lower spatio-temporal resolution: We use $N_{\rm{LES}}$ that is 8 to 64 times smaller than $N_{\text{DNS}}$, and $\Delta t_{\text{LES}}=10\Delta t_{\text{DNS}}$.

For LES, equations are derived by applying a homogenous filter to leading to,

$$
\begin{aligned}
    \nabla^2\overline{\psi} &= -\overline{\omega}, \\
    \frac{\partial \overline{\omega}}{\partial t} + \mathcal{N}(\overline{\omega},\overline{\psi}) &= \frac{1}{Re} \nabla^2\overline{\omega} -\chi\overline{\omega} - \overline{f} -\left(\overline{\mathcal{N}(\omega,\psi)}-\mathcal{N}(\overline{\omega},\overline{\psi}) \right), 
\end{aligned}
$$

$$
\begin{aligned}
    \Pi_{\omega} &= \nabla \times \nabla \cdot \bm{\tau} \\
    &= \overline{\mathcal{N}(\omega,\psi)}-\mathcal{N}(\overline{\omega},\overline{\psi})
\end{aligned}
$$


## Eddy Viscosity Models
$$
\Pi_{\omega} = - \nu_e \nabla^2 \overline{\omega},
$$
where $\nu$ is the eddy viscosity needs to be modeled

### Smagorinsky (SMAG)
$$
\nu_e = (C_s\Delta)^2 |\overline{\mathcal{S}}| \\
\overline{\mathcal{S}} = \sqrt{2 \overline{S}_{ij} \overline{S}_{ij}} \\
\overline{S}_{ij} = \frac{1}{2} \left(\frac{\partial \overline{u}_i}{\partial x_j} + \frac{\partial \overline{u}_j}{\partial x_i} \right)
$$
Here $C_s$ is the SMAG coefficient or eddy viscosity coefficient and $\Delta$ is the filter width.

### LEITH
$$
\nu_e = (C_L\Delta)^3 |\nabla \overline{\omega}| \\
$$
Here $C_L$ is the LEITH coefficient or eddy viscosity coefficient. 

In [2]:
# Import the Py2D_solver function from the py2d.Py2D_solver module
from py2d.Py2D_solver import Py2D_solver
# Import the numpy library for numerical operations
import numpy as np

# The code saves the snapshots of Omega in the folder: result/

# Initialize a flag to control whether to resume a simulation
resumeSimulation = False
# Set an initial value for the eddy viscosity coefficient
eddyViscosityCoeff_temp = 0.17

# Loop through 10 iterations to update the simulation
for count in range(10):

    # For iterations after the first one
    if count > 0:
        # Set the flag to resume the simulation from the last saved state
        resumeSimulation = True

        # Update the eddy viscosity coefficient after running the solver for a few iterations
        # Here, it's simply incremented by 0.01 from its initial value
        eddyViscosityCoeff_temp = 0.17 + 0.01

    # Call the Py2D_solver function with specified parameters
    Omega = Py2D_solver(
        Re=20e3,  # Reynolds number
        fkx=4,  # Forcing wavenumber in x
        fky=4,  # Forcing wavenumber in y
        alpha=0.1,  # Rayleigh drag coefficient
        beta=0,  # Coriolis parameter
        NX=128,  # Number of grid points in x and y (options: 32, 64, 128, 256, 512)
        SGSModel_string='LEITH',  # SGS model to use (options: 'NoSGS', 'SMAG', 'DSMAG', 'LEITH', etc.)
        eddyViscosityCoeff=eddyViscosityCoeff_temp,  # Coefficient for eddy viscosity models
        dt=5e-4,  # Time step
        saveData=True,  # Flag to save data
        tSAVE=0.001,  # Time interval to save data
        tTotal=0.01,  # Total time of simulation
        readTrue=False, 
        ICnum=1,  # Initial condition number (options: 1 to 20)
        resumeSim=resumeSimulation,  # Flag to start a new simulation or resume an existing one
    )

    # Print the domain average of enstrophy, calculated as the mean of the square of Omega
    print(np.mean(Omega**2))


[Errno 17] File exists: 'results/Re20000_fkx4fky4_r0.1_b0/LEITH/NX128/dt0.0005_IC1/data/'
+--------------------------------------------------------------------+--------+
|                         Run Configuration                          | Value  |
+--------------------------------------------------------------------+--------+
|                           Time Step (dt)                           | 0.0005 |
|                         Resume Simulation                          | False  |
| Read Initialization (readTrue), If False: Will read IC from a file | False  |
|                      Saving Data  (saveData)                       |  True  |
|               Save data every t th timestep (tSAVE)                | 0.001  |
|               Save data every Nth iteration (NSAVE)                |   2    |
|                   Length of simulation (tTotal)                    |  0.01  |
|                Maximum Number of Iterations (maxit)                |   20   |
+-----------------------------